In [1]:
import gym
import itertools
import numpy as np
import os
import random
import sys
import tensorflow as tf

if "../" not in sys.path:
  sys.path.append("../")

from lib import plotting
from collections import deque, namedtuple

In [2]:
env = gym.envs.make("Breakout-v0")

# Considered Actions: 0 (noop), 1 (fire), 2 (left) and 3 (right) 
VALID_ACTIONS = [0, 1, 2, 3]

[2016-12-15 22:29:57,354] Making new env: Breakout-v0


In [3]:
class StateProcessor():
    """
    Processes a raw Atari iamges. Resizes it and converts it to grayscale.
    """
    def __init__(self):
        # Build the Tensorflow graph
        with tf.variable_scope("state_processor"):
            self.input_state = tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
            self.output = tf.image.rgb_to_grayscale(self.input_state)
            self.output = tf.image.crop_to_bounding_box(self.output, 34, 0, 160, 160)
            self.output = tf.image.resize_images(
                self.output, [84, 84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
            self.output = tf.squeeze(self.output)

    def process(self, sess, state):
        """
        Argument:
            sess: Tensorflow session object
            state: Image [210, 160, 3] input dimension and rgb channels.

        Returns:
            Processed [84, 84, 1] grayscale values representing state.
        """
        return sess.run(self.output, { self.input_state: state })

In [4]:
class Estimator():
    """
    Q-Value Estimator neural network.

    This network is used for both Q-Network(action selection) and Target Network(policy evalutaion).
    """

    def __init__(self, scope="estimator", summaries_dir=None):
        self.scope = scope
        # Tensorboard summary write
        self.summary_writer = None
        with tf.variable_scope(scope):
            # Building the graph
            self._build_model()
            # Writing summaries to directory
            if summaries_dir:
                summary_dir = os.path.join(summaries_dir, "summaries_{}".format(scope))
                if not os.path.exists(summary_dir):
                    os.makedirs(summary_dir)
                self.summary_writer = tf.summary.FileWriter(summary_dir)


    def _build_model(self):
        """
        Builds the Tensorflow graph.
        """

        # Input: 4 frames of shape 84 * 84 each
        self.X_pl = tf.placeholder(shape=[None, 84, 84, 4], dtype=tf.uint8, name="X")
        # The target value
        self.y_pl = tf.placeholder(shape=[None], dtype=tf.float32, name="y")
        # Id which action was selected
        self.actions_pl = tf.placeholder(shape=[None], dtype=tf.int32, name="actions")

        X = tf.to_float(self.X_pl) / 255.0
        batch_size = tf.shape(self.X_pl)[0]

        # first convolutional layer- input [84 * 84 * 4], filter [8 * 8], num_filters 32, stride 4
        conv1 = tf.contrib.layers.conv2d(X, 32, 8, 4, activation_fn=tf.nn.relu)
        # second convolutional layer- input [20 * 20 * 32], filter [4 * 4], num_filters 64, stride 2
        conv2 = tf.contrib.layers.conv2d(conv1, 64, 4, 2, activation_fn=tf.nn.relu)
        # third convolutional layer- input [9 * 9 * 64], filter [3 * 3], num_filters 64, stride 1
        conv3 = tf.contrib.layers.conv2d(conv2, 64, 3, 1, activation_fn=tf.nn.relu)

        # first fully connected layer- input [7 * 7 * 64], num_filters 512
        flattened = tf.contrib.layers.flatten(conv3)
        fc1 = tf.contrib.layers.fully_connected(flattened, 512)
        # seconf fully connexted layer- input 512, num_filters 4, linear
        self.predictions = tf.contrib.layers.fully_connected(fc1, len(VALID_ACTIONS))

        # getting the predictions for the chosen actions only
        gather_indices = tf.range(batch_size) * tf.shape(self.predictions)[1] + self.actions_pl
        self.action_predictions = tf.gather(tf.reshape(self.predictions, [-1]), gather_indices)

        # calcualting the loss
        self.losses = tf.squared_difference(self.y_pl, self.action_predictions)
        self.losses = tf.clip_by_value(self.losses, -1, 1)
        self.loss = tf.reduce_mean(self.losses)

        # Passing Optimizer parameters from original paper
        self.optimizer = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6)
        self.train_op = self.optimizer.minimize(self.loss, global_step=tf.contrib.framework.get_global_step())

        # Tensorboard summaries
        self.summaries = tf.summary.merge([
            tf.summary.scalar("loss", self.loss),
            tf.summary.histogram("loss_hist", self.losses),
            tf.summary.histogram("q_values_hist", self.predictions),
            tf.summary.scalar("max_q_value", tf.reduce_max(self.predictions))
        ])


    def predict(self, sess, state):
        """
        Predicts action values.

        Arguments:
          sess: Tensorflow session
          state: State input of shape [batch_size, 4, 84, 84, 3]

        Returns:
          Tensor of shape [batch_size, NUM_VALID_ACTIONS] containing the estimated action values.
        """
        return sess.run(self.predictions, { self.X_pl: state })

    def update(self, sess, state, a, y):
        """
        Updates the estimator towards the given targets.

        Argumentss:
          sess: Tensorflow session object
          state: Image of shape [batch_size, 4, 84, 84, 3]
          a: Chosen actions of shape [batch_size]
          y: Targets of shape [batch_size]

        Returns:
          The calculated loss on the batch.
        """
        feed_dict = { self.X_pl: state, self.y_pl: y, self.actions_pl: a }
        summaries, global_step, _, loss = sess.run(
            [self.summaries, tf.contrib.framework.get_global_step(), self.train_op, self.loss],
            feed_dict)
        
        if self.summary_writer:
            self.summary_writer.add_summary(summaries, global_step)
        return loss

In [5]:
def copy_model_parameters(sess, estimator1, estimator2):
    """
    Copies the model parameters of one estimator to another.

    Arguments:
      sess: Tensorflow session instance
      estimator1: Estimator to copy the paramters from
      estimator2: Estimator to copy the parameters to
    """
    e1_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator1.scope)]
    e1_params = sorted(e1_params, key=lambda v: v.name)
    e2_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator2.scope)]
    e2_params = sorted(e2_params, key=lambda v: v.name)

    update_ops = []
    for e1_v, e2_v in zip(e1_params, e2_params):
        op = e2_v.assign(e1_v)
        update_ops.append(op)

    sess.run(update_ops)

In [6]:
def make_epsilon_greedy_policy(estimator, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.

    Arguments:
        estimator: An estimator that returns q values for a given state
        nA: Number of actions in the environment.

    Returns:
        A function that takes the (sess, observation, epsilon) as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.

    """
    def policy_fn(sess, observation, epsilon):
        
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(sess, np.expand_dims(observation, 0))[0]
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [ ]:
def deep_q_learning(sess,
                    env,
                    q_estimator,
                    target_estimator,
                    state_processor,
                    num_episodes,
                    experiment_dir,
                    replay_memory_size=500000,
                    replay_memory_init_size=50000,
                    update_target_estimator_every=10000,
                    discount_factor=0.99,
                    epsilon_start=1.0,
                    epsilon_end=0.1,
                    epsilon_decay_steps=500000,
                    batch_size=32,
                    record_video_every=50):
    """
    Q-Learning algorithm for fff-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.

    Arguments:
        sess: Tensorflow Session object
        env: OpenAI gym environment
        q_estimator: Estimator object used for the q values
        target_estimator: Estimator object used for the targets
        state_processor: StateProcessor object
        num_episodes: Number of episodes to run for
        experiment_dir: Directory to save Tensorflow summaries in
        replay_memory_size: Size of the replay memory
        replay_memory_init_size: Number of random experiences to sample when initializing the reply memory.
        update_target_estimator_every: Copy parameters from the Q estimator to the target estimator every N steps
        discount_factor: Lambda time discount factor
        epsilon_start: Chance to sample a random action when taking an action. Epsilon is decayed over time and this is the start value
        epsilon_end: The final minimum value of epsilon after decaying is done
        epsilon_decay_steps: Number of steps to decay epsilon over
        batch_size: Size of batches to sample from the replay memory
        record_video_every: Record a video every N episodes

    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """

    Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])

    # replay memory
    replay_memory = []

    # keeping track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))

    # create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    checkpoint_path = os.path.join(checkpoint_dir, "model")
    monitor_path = os.path.join(experiment_dir, "monitor")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(monitor_path):
        os.makedirs(monitor_path)

    saver = tf.train.Saver()
    # load a previous checkpoint if avalilable
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)

    total_t = sess.run(tf.contrib.framework.get_global_step())

    # epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)

    # our game policy
    policy = make_epsilon_greedy_policy(q_estimator, len(VALID_ACTIONS))

    # initializing replay memory with initial experiences
    print("Initializing replay memory.")
    state = env.reset()
    state = state_processor.process(sess, state)
    state = np.stack([state] * 4, axis=2)
    
    for i in range(replay_memory_init_size):
        
        action_probs = policy(sess, state, epsilons[min(total_t, epsilon_decay_steps-1)])
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
        next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
        next_state = state_processor.process(sess, next_state)
        next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
        replay_memory.append(Transition(state, action, reward, next_state, done))
        if done:
            state = env.reset()
            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)
        else:
            state = next_state

    # recording game video
    env.monitor.start(monitor_path,
                      resume=True,
                      video_callable=lambda count: count % record_video_every == 0)

    for i_episode in range(num_episodes):

        # saving the current checkpoint
        saver.save(tf.get_default_session(), checkpoint_path)

        # resetting the environment
        state = env.reset()
        state = state_processor.process(sess, state)
        state = np.stack([state] * 4, axis=2)
        loss = None

        for t in itertools.count():

            # epsilon for this time step
            epsilon = epsilons[min(total_t, epsilon_decay_steps-1)]

            # adding epsilon to Tensorboard
            episode_summary = tf.Summary()
            episode_summary.value.add(simple_value=epsilon, tag="epsilon")
            q_estimator.summary_writer.add_summary(episode_summary, total_t)

            # updating the target estimator
            if total_t % update_target_estimator_every == 0:
                copy_model_parameters(sess, q_estimator, target_estimator)
                print("\nCopied model parameters to target network.")

            # printing useful information for debugging
            print("\rStep {} ({}) @ Episode {}/{}, loss: {}".format(
                    t, total_t, i_episode + 1, num_episodes, loss), end="")
            sys.stdout.flush()

            # taking a step
            action_probs = policy(sess, state, epsilon)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
            next_state = state_processor.process(sess, next_state)
            next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)

            # saving latest transition to replay memory
            if len(replay_memory) == replay_memory_size:
                replay_memory.pop(0)
            replay_memory.append(Transition(state, action, reward, next_state, done))   

            # updating statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t

            # sampling a minibatch from the replay memory
            samples = random.sample(replay_memory, batch_size)
            states_batch, action_batch, reward_batch, next_states_batch, done_batch = map(np.array, zip(*samples))

            # calculating q values and targets (Double DQN)
            q_values_next = q_estimator.predict(sess, next_states_batch)
            best_actions = np.argmax(q_values_next, axis=1)
            q_values_next_target = target_estimator.predict(sess, next_states_batch)
            targets_batch = reward_batch + np.invert(done_batch).astype(np.float32) * \
                discount_factor * q_values_next_target[np.arange(batch_size), best_actions]

            # performing gradient descent update
            states_batch = np.array(states_batch)
            loss = q_estimator.update(sess, states_batch, action_batch, targets_batch)

            # shows an episode has terminated when set to true
            if done:
                break

            state = next_state
            total_t += 1

        # adding summaries to Tensorboard
        episode_summary = tf.Summary()
        episode_summary.value.add(simple_value=stats.episode_rewards[i_episode], node_name="episode_reward", tag="episode_reward")
        episode_summary.value.add(simple_value=stats.episode_lengths[i_episode], node_name="episode_length", tag="episode_length")
        q_estimator.summary_writer.add_summary(episode_summary, total_t)
        q_estimator.summary_writer.flush()

        yield total_t, plotting.EpisodeStats(
            episode_lengths=stats.episode_lengths[:i_episode+1],
            episode_rewards=stats.episode_rewards[:i_episode+1])

    env.monitor.close()
    return stats


In [ ]:
tf.reset_default_graph()

# directory to save our checkpoints and graphs
experiment_dir = os.path.abspath("./experiments-100/{}".format(env.spec.id))

# creating a global step variable
global_step = tf.Variable(0, name='global_step', trainable=False)

# creating both the estimators
q_estimator = Estimator(scope="q", summaries_dir=experiment_dir)
target_estimator = Estimator(scope="target_q")

# initializing state processor
state_processor = StateProcessor()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t, stats in deep_q_learning(sess,
                                    env,
                                    q_estimator=q_estimator,
                                    target_estimator=target_estimator,
                                    state_processor=state_processor,
                                    experiment_dir=experiment_dir,
                                    num_episodes=100,
                                    replay_memory_size=50000,
                                    replay_memory_init_size=5000,
                                    update_target_estimator_every=1000,
                                    epsilon_start=1.0,
                                    epsilon_end=0.1,
                                    epsilon_decay_steps=500000,
                                    discount_factor=0.75,
                                    batch_size=32):

        print("\nEpisode Reward: {}".format(stats.episode_rewards[-1]))
        np.save('Episode_Reward_%s' %t, stats)


/Users/leena/anaconda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Initializing replay memory.


[2016-12-15 22:32:16,401] Starting new video recorder writing to /Users/leena/Documents/AI/reinforcement-learning/DQN/experiments-100/Breakout-v0/monitor/openaigym.video.0.4011.video000000.mp4



Copied model parameters to target network.
Step 250 (250) @ Episode 1/100, loss: 0.06256594508886337
Episode Reward: 2.0
Step 360 (610) @ Episode 2/100, loss: 0.0003187861875630915
Episode Reward: 3.0
Step 177 (787) @ Episode 3/100, loss: 0.000272589735686779
Episode Reward: 0.0
Step 172 (959) @ Episode 4/100, loss: 0.0009831669740378857
Episode Reward: 0.0
Step 40 (999) @ Episode 5/100, loss: 0.0006929963710717857
Copied model parameters to target network.
Step 313 (1272) @ Episode 5/100, loss: 0.00022475063451565802
Episode Reward: 3.0
Step 261 (1533) @ Episode 6/100, loss: 1.7335725715383887e-05
Episode Reward: 2.0
Step 181 (1714) @ Episode 7/100, loss: 6.129081884864718e-05
Episode Reward: 0.0
Step 172 (1886) @ Episode 8/100, loss: 9.596770541975275e-05
Episode Reward: 0.0
Step 113 (1999) @ Episode 9/100, loss: 0.00011316889140289277
Copied model parameters to target network.
Step 307 (2193) @ Episode 9/100, loss: 2.6690962840802968e-05
Episode Reward: 2.0
Step 329 (2522) @ Episod

[2016-12-16 02:19:40,425] Starting new video recorder writing to /Users/leena/Documents/AI/reinforcement-learning/DQN/experiments-100/Breakout-v0/monitor/openaigym.video.0.4011.video000050.mp4


Step 184 (13275) @ Episode 51/100, loss: 1.696057734079659e-05
Episode Reward: 0.0
Step 171 (13446) @ Episode 52/100, loss: 3.1886818760540336e-05
Episode Reward: 0.0
Step 168 (13614) @ Episode 53/100, loss: 5.614896508632228e-05
Episode Reward: 0.0
Step 159 (13773) @ Episode 54/100, loss: 0.00012172339484095573
Episode Reward: 0.0
Step 226 (13999) @ Episode 55/100, loss: 0.00016653320926707238
Copied model parameters to target network.
Step 245 (14018) @ Episode 55/100, loss: 6.69665023451671e-05
Episode Reward: 1.0
Step 171 (14189) @ Episode 56/100, loss: 0.030586162582039833
Episode Reward: 0.0
Step 177 (14366) @ Episode 57/100, loss: 0.03106698766350746
Episode Reward: 0.0
Step 203 (14569) @ Episode 58/100, loss: 5.3500749345403165e-05
Episode Reward: 1.0
Step 287 (14856) @ Episode 59/100, loss: 0.031656090170145035